In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00


In [ ]:
from IPython.display import clear_output
!pip install transformers datasets
!pip install --upgrade transformers

clear_output()

In [ ]:
import pandas as pd

In [ ]:

data_path = "Cincy Dataset Final.xlsx"
dataset = pd.read_excel(data_path)

In [ ]:
dataset.columns = ['completion', 'prompt'] + list(dataset.columns[2:])


In [ ]:
df = dataset

In [ ]:
# Convert the 'prompt' and 'completion' columns to strings
df['prompt'] = df['prompt'].astype(str)
df['completion'] = df['completion'].astype(str)

tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Encode the prompts and completions using the tokenizer
input_encoding = tokenizer.batch_encode_plus(df['prompt'].values.tolist(), padding=True, return_tensors='pt')
output_encoding = tokenizer.batch_encode_plus(df['completion'].values.tolist(), padding=True, return_tensors='pt')


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

# Load the dataframe
# df = pd.read_csv('data.csv')
# df['completion'] = df['completion'].astype(str).str.replace('\n', ' ##')
# # Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Encode the prompts and completions using the tokenizer
input_encoding = tokenizer.batch_encode_plus(df['prompt'].values.tolist(), padding=True, return_tensors='pt')
output_encoding = tokenizer.batch_encode_plus(df['completion'].values.tolist(), padding=True, return_tensors='pt')

# Extract the input_ids and attention_mask values from the encoding dictionaries
input_ids = input_encoding['input_ids']
input_mask = input_encoding['attention_mask']
output_ids = output_encoding['input_ids']
output_mask = output_encoding['attention_mask']

# Create a list of dictionaries containing the input_ids and attention_mask values
train_data = [{'input_ids': input_ids[i], 'attention_mask': input_mask[i], 'labels': output_ids[i]} for i in range(len(input_ids))]
eval_data = [{'input_ids': output_ids[i], 'attention_mask': output_mask[i], 'labels': output_ids[i]} for i in range(len(output_ids))]

# Initialize the T5 model for sequence-to-sequence learning
model = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=300,
    evaluation_strategy='steps',
    save_total_limit=2,
    eval_steps=300,
    save_steps=1000,
    learning_rate=5e-5,
)

# Define the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Fine-tune the model
trainer.train()


Step,Training Loss,Validation Loss
300,4.656700,0.031885
600,0.277900,0.015697
900,0.188900,0.012681
1200,0.142300,0.010860


Step,Training Loss,Validation Loss
300,4.656700,0.031885
600,0.277900,0.015697
900,0.188900,0.012681
1200,0.142300,0.010860
1500,0.109800,0.009288
1800,0.091500,0.009323
2100,0.077100,0.009489
2400,0.069600,0.008629
2700,0.062100,0.008772
3000,0.058800,0.008893


TrainOutput(global_step=3220, training_loss=0.5381699402139793, metrics={'train_runtime': 1462.9269, 'train_samples_per_second': 21.942, 'train_steps_per_second': 2.201, 'total_flos': 4619635435008000.0, 'train_loss': 0.5381699402139793, 'epoch': 20.0})

In [ ]:
model.save_pretrained("./cincy_model_20epoch")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%mv /content/cincy_model_20epoch  /content/gdrive/MyDrive/cincy_model_20epoch

Mounted at /content/gdrive


In [ ]:
!mkdir data

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = 't5-base'
model_save_path = "data/model_v1"
# Load the dataframe
df = pd.read_csv('/content/data/chap_gpt_with_dates.csv')
df['completion'] = df['completion'].astype(str).str.replace('\n', ' ##')

train = df.sample(1400)
test = df[~df["prompt"].isin(train["prompt"])]


train.to_csv("data/train_set.csv",index=False)
test.to_csv("data/test_set.csv",index=False)

print(train.shape)
print(test.shape)
# Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained(model)
# Initialize the T5 model for sequence-to-sequence learning
model = T5ForConditionalGeneration.from_pretrained(model).to(device)

# Encode the prompts and completions using the tokenizer
train_input_encoding = tokenizer.batch_encode_plus(train['prompt'].tolist(), padding=True, return_tensors='pt')
train_output_encoding = tokenizer.batch_encode_plus(train['completion'].tolist(), padding=True, return_tensors='pt')

eval_input_encoding = tokenizer.batch_encode_plus(test['prompt'].tolist(), padding=True, return_tensors='pt')
eval_output_encoding = tokenizer.batch_encode_plus(test['completion'].tolist(), padding=True, return_tensors='pt')

# Extract the input_ids and attention_mask values from the encoding dictionaries
# input_ids = input_encoding['input_ids']
# input_mask = input_encoding['attention_mask']
# output_ids = output_encoding['input_ids']
# output_mask = output_encoding['attention_mask']

# print(len(input_ids), len(output_ids))

# Create a list of dictionaries containing the input_ids and attention_mask values
train_data = [{'input_ids': train_input_encoding['input_ids'][i],
               'attention_mask': train_input_encoding['attention_mask'][i],
               'labels': train_output_encoding['input_ids'][i]} for i in
              range(len(train_input_encoding['input_ids']))]
eval_data = [{'input_ids': eval_input_encoding['input_ids'][i],
              'attention_mask': eval_input_encoding['attention_mask'][i],
              'labels': eval_output_encoding['input_ids'][i]} for i in
             range(len(eval_input_encoding['input_ids']))]

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy='steps',
    save_total_limit=2,
    eval_steps=1000,
    save_steps=1000,
    learning_rate=5e-5,
)

# Define the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Fine-tune the model
trainer.train()
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


(1400, 2)
(411, 2)


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1400
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1400
  Number of trainable parameters = 222903552


Step,Training Loss,Validation Loss
1000,1.376700,0.065266


***** Running Evaluation *****
  Num examples = 411
  Batch size = 10
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Configuration saved in ./results/checkpoint-1000/generation_config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in data/model_v1/config.json
Configuration saved in data/model_v1/generation_config.json
Model weights saved in data/model_v1/pytorch_model.bin
tokenizer config file saved in data/model_v1/tokenizer_config.json
Special tokens file saved in data/model_v1/special_tokens_map.json


('data/model_v1/tokenizer_config.json',
 'data/model_v1/special_tokens_map.json',
 'data/model_v1/spiece.model',
 'data/model_v1/added_tokens.json')

In [ ]:
model.save_pretrained("./model")

Configuration saved in ./model/config.json
Configuration saved in ./model/generation_config.json
Model weights saved in ./model/pytorch_model.bin


In [ ]:
from tqdm import tqdm

In [ ]:
df

,prompt,completion
0,"""As the results of all three tests for Telecom...",20% -- threshold ##20% -- significance
1,$10.30 per Public Share or (ii) the actual amo...,$10.30 -- Public Share ##$10.30 -- Public Share
2,"$120,000,000 (the “Base Price”); plus (b) the ...","$120,000,000 -- Base Price"
3,$148.3M - $148.9M in estimated Rubraca product...,$ 148.3 M - $ 148.9 -- US and Europe product r...
4,"$18.799 in respect of the Pre-Funded Warrants,...",$18.799 -- respect of the Pre-Funded Warrants ...
...,...,...
1803,"•Total revenue, excluding COVID-19 testing, in...",29 percent -- total revenue increased
1804,•Unaudited preliminary fourth quarter 2021 ING...,$ 301 million and $ 296 million -- net sales
1805,•We expect fourth quarter 2021 non-GAAP operat...,28% -- non-GAAP operating margin
1806,•We expect full year 2021 non-GAAP operating m...,30% -- non-GAAP operating margin


In [ ]:
# Create an empty list to store the predictions
predictions = []

# Loop over each row in the input data, displaying a progress bar using tqdm
for index, row in tqdm(df.iterrows(), total=len(df)):

    # Get the prompt text from the current row
    prompt = row["prompt"]

    # Generate a completion for the prompt using the T5 model
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output_ids = model.generate(input_ids, max_length=100).to("cuda")
    completion = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Append the completion to the list of predictions
    predictions.append(completion)



# Add the predictions as a new column to the DataFrame
df["predictions"] = predictions

# Save the updated DataFrame to a new CSV file
df.to_csv("output_10epoch.csv", index=False)


Streaming output truncated to the last 5000 lines.
 61%|██████    | 1094/1808 [08:40<05:36,  2.12it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

 61%|██████    | 1095/1808 [08:40<04:43,  2.51it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

 61%|██████    | 1096/1808 [08:41<04:33,  2.60it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

 61%|██████    | 1097/1808 [08:41<03:48,  3.12it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

 61%|██████    | 1098/1808 [08:41<03:37,  3.26it/s]Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%mv /content/data  /content/gdrive/MyDrive/data_modelv1

Mounted at /content/gdrive


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained T5 model
#model = T5ForConditionalGeneration.from_pretrained("./model")

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Prompt for user input
# prompt = input("ask")
prompt = 'â€¢We expect full year 2021 non-GAAP operating margin of nearly 30%'

# Encode the prompt using the tokenizer
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate the completion
output = model.generate(
    input_ids=input_ids,
    max_length=50,
    num_beams=4,
    no_repeat_ngram_size=2,
    early_stopping=True
)


# Decode the completion using the tokenizer
completion = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the completion


clear_output()

print('question: {}'.format(prompt))
print('answer: {}'.format(completion))


loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_p

RuntimeError: ignored

question: $8 million Royalty Convertible Debenture: 9.0% debt convertible to 1.85% life of mine royalty to fund mine purchase, initial EPA payment and on-going engineering. Closing expected in January 2022
answer: $8 million -- Royalty Convertible Debenture 9.0% -- debt convertible 1.85% -- life of mine royalty
